# FileOrganizer Antonio Furioso

Inizio con l'importare tutte le librerie che sono necessarie per lo step 1 e 3

In [1]:
import os, csv, shutil
import numpy as np
from PIL import Image
from tabulate import tabulate

# Step 1

Nel seguente step devo definire il tipo di file e prima di spostarlo nella cartella che determina la sua categoria (che deve essere creata automaticamente), devo stampare e scrivere le sue informazioni su file di recap.csv

In [2]:
'''definisco una funzione per conoscere il tipo di file'''
def tipo_file(filename):
    
    #Verifico se il file non è una cartella
    if os.path.isdir(filename) == False:

        #per ogni tipo di file indico le sue possibili estensioni
        doc = ['.doc', '.txt', '.odt', '.docx', '.pdf']
        image = ['.jpg','.jpeg','.png','.gif']
        audio = ['.mp3','.wav']
        
        #utilizzo split per individuare l'estensione dei file
        if os.path.splitext(filename)[1] in image:
            return 'image'
        elif os.path.splitext(filename)[1] in doc:
            return 'doc'
        elif os.path.splitext(filename)[1] in audio:
            return 'audio'
        else: #se il file non è in un formato leggibile o non elencato allora restituisce un errore
            return 'Non leggibile'

Creazione di una cartella in base al tipo di file:

In [3]:
for file in os.listdir('C:\\Users\\anton\\Desktop\\FileOrganizer\\files\\'): #creo una cartella per ogni tipo di file
    if file != 'recap.csv' and tipo_file(file) != 'Non leggibile':   #se il file non è un recap.csv e non è un file leggibile allora creo la cartella
        try:
            os.mkdir('C:\\Users\\anton\\Desktop\\FileOrganizer\\files\\'+tipo_file(file))   #crea la cartella del tipo di file iterato se non esiste
        except:
            FileExistsError #restituiamo un errore nel caso le precendenti istruzioni non vengano confermate

Creiamo il file di recap.csv con prima riga che fa da header

In [4]:
if 'recap.csv' not in os.listdir('C:\\Users\\anton\\Desktop\\FileOrganizer\\files\\'): #se il file di recap non esiste allora ne crea uno
    with open ('C:\\Users\\anton\\Desktop\\FileOrganizer\\files\\recap.csv', 'w', newline='') as csvfile: #creiamo e apriamo il file per scriverci le info del file
        filewriter = csv.writer(csvfile, delimiter = ',', quotechar = '|')
        filewriter.writerow(['name', 'type', 'size(b)']) #questa sarà la prima riga del file


Infine possiamo procedere nel stampare le informazini, scriverle sul file di recap e spostare i files nelle relative sottocartelle create

In [8]:
for file in os.listdir('C:\\Users\\anton\\Desktop\\FileOrganizer\\files\\'):
    if tipo_file(file) in os.listdir('C:\\Users\\anton\\Desktop\\FileOrganizer\\files\\'):
        with open ('C:\\Users\\anton\\Desktop\\FileOrganizer\\files\\recap.csv', 'a', newline='') as csvfile: #aprimao il file così che sia pronto per la trascrizione
            filewriter = csv.writer(csvfile, delimiter = ',', quotechar = '|')
            print('name: '+ os.path.splitext(file)[0] + ' type: '+ tipo_file(file) + ' size: ' + str(os.path.getsize('C:\\Users\\anton\\Desktop\\FileOrganizer\\files\\'+file))+'B') #stampiamo le informazioni dei file
            filewriter.writerow([os.path.splitext(file)[0], tipo_file(file), str(os.path.getsize('C:\\Users\\anton\\Desktop\\FileOrganizer\\files\\' + file)) + 'B']) #inseriamo le informazioni dei file nel file di recap.csv
            shutil.move('C:\\Users\\anton\\Desktop\\FileOrganizer\\files\\' + file, 'C:\\Users\\anton\\Desktop\\FileOrganizer\\files\\' + tipo_file(file)) #spostiamo i file contenuti in files nelle relative cartelle

# Step 3

In questo step dobbiamo conoscere il colore e le dimensioni dei file nella cartella image (prima creata) e stamparle in tabella 

In [6]:
listadariempire= []   #apriamo una lista vuota nel quale inseriremo i dati

for image in os.listdir('C:\\Users\\anton\\Desktop\\FileOrganizer\\files\\image\\'): #prendiamo ogni immagine presente in cartella
    nomi = [os.path.splitext(image)[0]]   #per identificare l'iimmagine ci serve il suo nome che andremo ad inserire come prima colonna nella tabella
    img = Image.open('C:\\Users\\anton\\Desktop\\FileOrganizer\\files\\image\\'+image)   #apriamo le immagini utilizzando la libreria PIL
    imgwnp = np.array(img)   #con numpy trasformiamo le immagini in degli array
    vcolor = imgwnp.mean(axis=0).mean(axis=0) #prendiamo i valori dei colori presenti nelle immagini
    nomi.extend([imgwnp.shape[0],imgwnp.shape[1]])   #aggiungiamo le dimensioni
    if imgwnp.ndim == 2:   #assegnamo il numero di dimensioni 2 alla scala di grigio
        nomi.extend([vcolor,0.00,0.,0.,0])
    elif len(vcolor) == 3:   #assegnamo il numero di dimensioni 3 alla scala RGB
        nomi.extend([0.,vcolor[0],vcolor[1],vcolor[2],0])
    elif len(vcolor) == 4:   #assegnamo il numero di dimensioni 4 alla scala RGBA
        nomi.extend([0.,vcolor[0],vcolor[1],vcolor[2],vcolor[3]])
    listadariempire.append(nomi)   #aggiungiamo la lista a una lista

print(tabulate(listadariempire, headers=['name','height','width','grayscale','R','G','B','A'], floatfmt='.2f',tablefmt='fancy_grid')) #infine stampiamo e creiamo l'header della nostra tabella

╒══════════╤══════════╤═════════╤═════════════╤════════╤═══════╤═══════╤═════╕
│ name     │   height │   width │   grayscale │      R │     G │     B │   A │
╞══════════╪══════════╪═════════╪═════════════╪════════╪═══════╪═══════╪═════╡
│ bw       │      512 │     512 │       21.48 │   0.00 │  0.00 │  0.00 │   0 │
├──────────┼──────────┼─────────┼─────────────┼────────┼───────┼───────┼─────┤
│ daffodil │      500 │     335 │        0.00 │ 109.25 │ 85.56 │  4.97 │   0 │
├──────────┼──────────┼─────────┼─────────────┼────────┼───────┼───────┼─────┤
│ trump    │      183 │     275 │        0.00 │  97.01 │ 98.99 │ 90.92 │   0 │
╘══════════╧══════════╧═════════╧═════════════╧════════╧═══════╧═══════╧═════╛
